<a href="https://colab.research.google.com/github/alexvishnevskiy/PetFinder/blob/master/PetFinder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pytorch-lightning timm python-box -U albumentations wandb

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp /content/drive/MyDrive/PetFinder/petfinder-pawpularity-score.zip .
!unzip /content/petfinder-pawpularity-score.zip

In [75]:
from pytorch_lightning import LightningDataModule, LightningModule
from pytorch_lightning.utilities.seed import seed_everything
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning import callbacks
import pytorch_lightning as pl

from torch.utils.data import DataLoader, Dataset
from albumentations.pytorch.transforms import ToTensorV2
import albumentations as A

from timm import create_model
import torch.optim as optim
import torch.nn as nn
import torch

from sklearn.model_selection import StratifiedKFold
import pandas as pd
import numpy as np

import os
import cv2

from box import Box
import matplotlib.pyplot as plt
import wandb

In [ ]:
wandb.login()

In [68]:
cfg = {
    'seed': 42,
    'dir_path': '/content/',
    'photo_path': '/content/train',
    'csv_path': '/content/train.csv',
    'logger': {
        'save_dir': '/content/drive/MyDrive/PetFinder/models',
        'name': 'bert',
        'project': 'PetFinder',
        'log_model': True,
    },
    'transform': {
        'img_size': (224, 224),
        'train_transforms': None,
        'val_transforms': None,
    },
    'loader': {
        'train': {
            'batch_size': 64,
            'num_workers': 4,
            'shuffle': True,
            'pin_memory': False,
        },
        'val': {
            'batch_size': 64,
            'num_workers': 4,
            'shuffle': False,
            'pin_memory': False,
        }
    },
    'train_args': {
        'n_splits': 5,
        'epoch': 2,
        'find_lr': False,
    },
    'model': {
        'name': 'swin_tiny_patch4_window7_224',
        'alias_name': 'swin_tiny',
        'freeze_layers': 0,
        'dropout': 0.1,
        'output_dim': 1,
    },
    'loss': {
        'module': 'nn.BCEWithLogitsLoss',
        'alias': 'bce',
    },
    'optimizer':{
        'name': 'optim.AdamW',
        'params':{
            'lr': 1e-5,
            'weight_decay': 1e-4,
        },
    },
    'scheduler':{
        'name': 'optim.lr_scheduler.CosineAnnealingWarmRestarts',
        'params':{
            'T_0': 20,
            'eta_min': 1e-4,
        },
    },
    'trainer': {
        'gpus': 1,
        'accumulate_grad_batches': 1,
        'auto_lr_find': True,
        'progress_bar_refresh_rate': 3,
        'fast_dev_run': False,
        'num_sanity_val_steps': 2,
        #'overfit_batches': 1,
        'resume_from_checkpoint': None,
    },
    'results_callback': {
        'n_images': 30,
        'epoch': 1,
    },
}

cfg = Box(cfg)

In [69]:
seed_everything(cfg.seed)

Global seed set to 42


42

In [70]:
transforms = lambda img_size: A.Compose([
     A.Resize(*img_size),
     A.Normalize(
         mean = [0.485, 0.456, 0.406],
         std = [0.229, 0.224, 0.225],
         always_apply = True
         ),
     ToTensorV2(),
     ])

cfg.transform.train_transforms = transforms(cfg.transform.img_size)
cfg.transform.val_transforms = transforms(cfg.transform.img_size)

In [71]:
class CustomDataset(Dataset):
  def __init__(self, cfg, df: pd.DataFrame, stage: str):
    super().__init__()
    self.cfg = cfg
    self.df = self.prepare_df(df)
    self.stage = stage
    if stage == 'train':
      self.transforms = cfg.transform.train_transforms
    else:
      self.transforms = cfg.transform.val_transforms

  def prepare_df(self, df):
    df.loc[:, 'path'] = (
        df['Id']
        .apply(lambda x: os.path.join(self.cfg.photo_path, f'{x}.jpg'))
        )
    return df

  def __len__(self):
    return len(self.df)

  def __getitem__(self, index: int):
    photo_path = self.df.iloc[index]['path']
    label = self.df.iloc[index]['Pawpularity']

    img = self.prepare_img(photo_path)
    return img, label

  def prepare_img(self, path: str):
    _img = cv2.imread(path)
    _img = cv2.cvtColor(_img, cv2.COLOR_BGR2RGB)
    img = self.transforms(image=_img)['image']
    return img

In [72]:
class CustomDataModule(LightningDataModule):
  def __init__(self, cfg, train_df: pd.DataFrame, val_df: pd.DataFrame):
    super().__init__()
    self.cfg = cfg
    self.train_df = train_df
    self.val_df = val_df

  def train_dataloader(self):
    train_split = CustomDataset(self.cfg, self.train_df, 'train')
    return DataLoader(
        train_split,
        batch_size=self.cfg.loader.train.batch_size, 
        shuffle=self.cfg.loader.train.shuffle, 
        num_workers=self.cfg.loader.train.num_workers,
        )

  def val_dataloader(self):
    val_split = CustomDataset(self.cfg, self.val_df, 'val')
    return DataLoader(
        val_split, 
        batch_size=self.cfg.loader.val.batch_size, 
        shuffle=self.cfg.loader.val.shuffle,
        num_workers=self.cfg.loader.val.num_workers,
        )

In [73]:
class CustomModel(LightningModule):
  def __init__(self, cfg):
    super().__init__()
    self.cfg = cfg
    self.__build_model(cfg.model.freeze_layers)
    self._criterion = eval(self.cfg.loss.module)()
    self.save_hyperparameters(cfg)
    self.wandb_table = wandb.Table(columns=["Pawpularity", "pred", "image"])

  def __build_model(self, freeze_layers: int = 0):
    ## add freezing of layers
    self.backbone = create_model(
        self.cfg.model.name, pretrained=True, num_classes=0, in_chans=3
        )
    num_features = self.backbone.num_features
    self.fc = nn.Sequential(
        nn.Dropout(self.cfg.model.dropout),
        nn.Linear(num_features, self.cfg.model.output_dim)
        )
    
  def forward(self, x):
    f = self.backbone(x)
    out = self.fc(f)
    return out

  def configure_optimizers(self):
    optimizer = eval(self.cfg.optimizer.name)(
        self.parameters(), **self.cfg.optimizer.params
        )
    scheduler = eval(self.cfg.scheduler.name)(
        optimizer,
        **self.cfg.scheduler.params
        )
    return [optimizer], [scheduler]

  def __share_step(self, batch, stage = 'train'):
    img, labels = batch
    labels = labels.float()
    logits = self(img).squeeze()
    preds = logits.sigmoid()

    cond = isinstance(self._criterion, nn.BCEWithLogitsLoss)
    if cond:
      labels = labels / 100.0
      preds *= 100 
    
    loss = self._criterion(logits, labels)
    labels = labels*(100**cond)
    if stage == 'val':
      return loss, preds, labels, img
    return loss, preds, labels

  def __share_epoch(self, outputs, stage):
    preds = []
    labels = []
    for out in outputs:
      pred, label = out['pred'], out['labels']
      preds.append(pred)
      labels.append(label)
    preds = torch.cat(preds).cpu().detach()
    labels = torch.cat(labels).cpu().detach()
    rmse = torch.sqrt(((labels - preds) ** 2).mean())
    self.log(f'{stage}_rmse', rmse)

  def training_step(self, batch, batch_idx):
    loss, preds, labels = self.__share_step(batch, 'train')
    self.log('train_loss', loss)
    return {'loss': loss, 'pred': preds, 'labels': labels}
        
  def validation_step(self, batch, batch_idx):
    loss, preds, labels, img = self.__share_step(batch, 'val')
    self.log('val_loss', loss)

    if batch_idx % 5 == 0:
      for pr, l, im in zip(preds, labels, img):
        self.wandb_table.add_data(l, pr, wandb.Image(im.squeeze()))

    return {'loss': loss, 'pred': preds, 'labels': labels}

  def training_epoch_end(self, outputs):
    self.__share_epoch(outputs, 'train')

  def validation_epoch_end(self, outputs):
    self.__share_epoch(outputs, 'val')
    wandb.log({'results': self.wandb_table})
    self.wandb_table = wandb.Table(columns=["Pawpularity", "pred", "image"])

In [79]:
skf = StratifiedKFold(n_splits=cfg.train_args.n_splits, shuffle=True, random_state=cfg.seed)
df = pd.read_csv(cfg.csv_path)

In [ ]:
for fold, (train_idx, val_idx) in enumerate(skf.split(df["Id"], df["Pawpularity"])):
  print(f"{'*'*100}\n{'*'*45} fold: {fold} {'*'*46}\n{'*'*100}")

  train_df, val_df = df.loc[train_idx], df.loc[val_idx]
  datamodule = CustomDataModule(cfg, train_df, val_df)
  model = CustomModel(cfg)

  earystopping = EarlyStopping(monitor="val_loss")
  lr_monitor = callbacks.LearningRateMonitor()
  loss_checkpoint = callbacks.ModelCheckpoint(
      dirpath = cfg.logger.save_dir,
      filename=f"{cfg.model.alias_name}",
      monitor="val_rmse",
      save_top_k=1,
      mode="min",
      save_last=False,
      )

  wandb_logger = WandbLogger(
      name = f"fold_{fold}",
      project = cfg.logger.project,
      log_model = cfg.logger.log_model,
      reinit = True,
      group = f"{cfg.logger.name}",
      id = f"{cfg.model.alias_name}_fold{fold}",
      )
  wandb_logger.watch(model)
      
  trainer = pl.Trainer(
      max_epochs=cfg.train_args.epoch,
      logger = wandb_logger,
      callbacks=[lr_monitor, loss_checkpoint, earystopping],
      deterministic=True,
      **cfg.trainer,
      )
  
  trainer.fit(model, datamodule=datamodule)
  wandb.finish()

In [ ]:
# if cfg.train_args.find_lr:
#   lr_finder = trainer.tuner.lr_find(model, datamodule=datamodule)
#   print(lr_finder.suggestion())
#   fig = lr_finder.plot(suggest=True)
#   fig.show()

In [ ]:
#cfg.optimizer.params.lr = 3e-4
#cfg.train_args.find_lr = False